In [1]:
#for enable TPU
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4139  100  4139    0     0  19803      0 --:--:-- --:--:-- --:--:-- 19803
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Found existing installation: torch 1.5.0
Uninstalling torch-1.5.0:
  Successfully uninstalled torch-1.5.0
Found existing installation: torchvision 0.6.0a0+35d732a
Uninstalling torchvision-0.6.0a0+35d732a:
Done updating TPU runtime
  Successfully uninstalled torchvision-0.6.0a0+35d732a
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl...

Operation completed over 1 objects/91.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl...

Operation completed over 1 objects/119.5 MiB.                                    
Copying gs://tpu-pytorch/wheels/to

In [2]:
!pip install colored

     |████████████████████████████████| 56 kB 1.8 MB/s 
  Created wheel for colored: filename=colored-1.4.2-py3-none-any.whl size=14002 sha256=c57c18a14c5721660754dedac98a82ca7caf26293e2ef4de5ed50866895d1ec1
  Stored in directory: /root/.cache/pip/wheels/5f/e1/fb/d0e85a8383ff58962319bb81c46e398fa1f4bb9e1feb0f81c4
Successfully built colored


In [3]:
!export XLA_USE_BF16=1

In [4]:
import gc
import time
import colored
import matplotlib.pyplot as plt
from colored import fg, bg, attr

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import os
import torch
import pandas as pd
from scipy import stats
import numpy as np

from tqdm import tqdm
from collections import OrderedDict, namedtuple
import torch.nn as nn
from torch.optim import lr_scheduler
import joblib

import logging
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule, XLMRobertaTokenizer, XLMRobertaModel, XLMRobertaConfig
from transformers import TFAutoModel, AutoTokenizer
import sys
from sklearn import metrics, model_selection

from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from transformers import RobertaModel, RobertaTokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences as pad
import warnings
import torch_xla
import torch_xla.debug.metrics as met
from torch import LongTensor, FloatTensor, DoubleTensor
from torch.utils.data import Dataset, DataLoader, sampler
from torch.utils.data.distributed import DistributedSampler
import torch_xla.distributed.data_parallel as dp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils
import warnings

warnings.filterwarnings("ignore")



wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [5]:
EPOCHS = 2
MAXLEN = 64
SPLIT = 0.8
DROP_RATE = 0.3
# LR = (4e-5, 1e-2)
LEARNING_RATE = 0.5e-5
BATCH_SIZE = 256
VAL_BATCH_SIZE = 512#8192
MODEL_SAVE_PATH = 'insincerity_model.pt'

In [6]:
np.random.seed(42)
torch.manual_seed(42)

In [7]:
# Dataset
train_df = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')

In [8]:
class QuoraDataset:
    def __init__(self, question_text, targets, tokenizer, max_length):
        self.question_text = question_text
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.targets = targets

    def __len__(self):
        return len(self.question_text)

    def __getitem__(self, item):
        
        question_text = str(self.question_text[item])
        question_text = " ".join(question_text.split())

        inputs = self.tokenizer.encode_plus(
            question_text,
            None,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation_strategy="longest_first",
            pad_to_max_length=True,
        )
        
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.targets[item], dtype=torch.float)
        }

In [9]:
class CustomRoberta(nn.Module):
    def __init__(self):
        super(CustomRoberta, self).__init__()
        self.num_labels = 1
        self.roberta = transformers.XLMRobertaModel.from_pretrained("xlm-roberta-base", output_hidden_states=False, num_labels=1)
        self.dropout = nn.Dropout(p=0.2)
        self.classifier = nn.Linear(768*2, self.num_labels)

    def forward(self,
                input_ids=None,
                attention_mask=None,
                position_ids=None,
                head_mask=None,
                inputs_embeds=None):

        o1, o2 = self.roberta(input_ids,
                               attention_mask=attention_mask,
                               position_ids=position_ids,
                               head_mask=head_mask,
                               inputs_embeds=inputs_embeds)
        
        apool = torch.mean(o1, 1)
        mpool, _ = torch.max(o1, 1)
        cat = torch.cat((apool, mpool), 1)
        bo = self.dropout(cat)
        logits = self.classifier(bo)       
        outputs = logits
        return outputs

In [10]:
m = CustomRoberta()
m

CustomRoberta(
  (roberta): XLMRobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [11]:
def bce(y_pred, y_true):
    return nn.BCEWithLogitsLoss()(y_pred, y_true.view(-1,1))*len(y_pred)

def f1_score(y_pred, y_true):
    y_true = y_true.squeeze()
    y_pred = torch.round(nn.Sigmoid()(y_pred)).squeeze()
    
    tp = (y_true * y_pred).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)
    tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    
    epsilon = 1e-7
    recall = tp / (tp + fn + epsilon)
    precision = tp / (tp + fp + epsilon)
    return 2*(precision*recall) / (precision + recall + epsilon)

In [12]:
global val_f1s; global train_f1s
global val_losses; global train_losses
def _run(train_df):
    tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
    split = np.int32(SPLIT*len(train_df))
    val_df, train_df = train_df[split:], train_df[:split]
    val_df = val_df.reset_index(drop=True)
    val_dataset = QuoraDataset(
        question_text = val_df.question_text.values,
        tokenizer = tokenizer,
        max_length = MAXLEN,
        targets = val_df.target.values
        )
    
    val_sampler = DistributedSampler(
        val_dataset, 
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(), 
        shuffle=True)
    
    val_loader = DataLoader(
        dataset=val_dataset, 
        batch_size=VAL_BATCH_SIZE,                    
        sampler=val_sampler, 
        num_workers=1, 
        drop_last=True)

    train_df = train_df.reset_index(drop=True)
    
    train_dataset = QuoraDataset(
        question_text = train_df.question_text.values,
        tokenizer = tokenizer,
        max_length = MAXLEN,
        targets = train_df.target.values
    )
    train_sampler = DistributedSampler(
        train_dataset, 
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(), 
        shuffle=True)

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        sampler=train_sampler, 
        num_workers=1, 
        drop_last=True)
    device = xm.xla_device()
    model = m.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    num_train_steps = int(len(train_dataset) / BATCH_SIZE / xm.xrt_world_size() * EPOCHS)
    xm.master_print(f'num_train_steps = {num_train_steps}, world_size={xm.xrt_world_size()}')
    optimizer = AdamW(
        optimizer_grouped_parameters, 
        lr=LEARNING_RATE * xm.xrt_world_size()
    )
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_train_steps
    )
    val_losses = []
    train_losses = [] 
    val_f1s = [] 
    train_f1s = []
    start = time.time()
    xm.master_print("STARTING TRAINING ...\n")
    for epoch in range(EPOCHS):
        fonts = (fg(48), attr('reset'))
        xm.master_print(("EPOCH %s" + str(epoch+1) + "%s") % fonts)
        val_parallel = pl.ParallelLoader(val_loader, [device]).per_device_loader(device)
        train_parallel = pl.ParallelLoader(train_loader, [device]).per_device_loader(device)
        
        batch = 1
        model.train()
        for bi, d in enumerate(train_parallel):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["targets"]
            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)
            outputs = model(
                input_ids=ids,
                attention_mask=mask,
            )
            
            train_loss = bce(outputs, targets)/len(outputs)
            train_f1 = f1_score(outputs, targets)
            
            optimizer.zero_grad()
            train_loss.backward()
            xm.optimizer_step(optimizer)
            end = time.time()
            f1 = np.round(train_f1.item(), 3)
            if bi % 100 == 0:
                 xm.master_print(f'Train bi={bi}, F1={f1},time={end-start}')
    
        val_loss = 0
        val_f1 = 0
        val_points = 0
        model.eval()
        with torch.no_grad():
            for bi, d in enumerate(val_parallel):
                ids = d["ids"]
                mask = d["mask"]
                targets = d["targets"]
                ids = ids.to(device, dtype=torch.long)
                mask = mask.to(device, dtype=torch.long)
                targets = targets.to(device, dtype=torch.float)

                outputs = model(
                    input_ids=ids,
                    attention_mask=mask,
                )
                val_points += len(targets)
                val_loss += bce(outputs, targets)
                val_f1 += f1_score(outputs, targets)*len(outputs)
        end = time.time()
        val_f1 /= val_points
        val_loss /= val_points
        f1 = xm.mesh_reduce('f1', val_f1, lambda x: sum(x)/len(x))
        loss = xm.mesh_reduce('loss', val_loss, lambda x: sum(x)/len(x))
        xm.master_print(f'val bi={bi}, F1={f1},time={end-start}')
        xm.master_print("")
        val_f1s.append(f1); train_f1s.append(train_f1.item())
        val_losses.append(loss); train_losses.append(train_loss.item())
    xm.master_print("ENDING TRAINING ...")
    xm.save(model.state_dict(), MODEL_SAVE_PATH)
    del model
    gc.collect()
    metric_lists = [val_losses, train_losses, val_f1s, train_f1s]
    metric_names = ['val_loss_', 'train_loss_', 'val_f1_', 'train_f1_']
    
    for i, metric_list in enumerate(metric_lists):
        for j, metric_value in enumerate(metric_list):
            torch.save(metric_value, metric_names[i] + str(j) + '.pt')

In [13]:
FLAGS = {}
train_df = shuffle(train_df)
train_df = train_df.reset_index(drop=True)

def _mp_fn(rank, flags):
    torch.set_default_tensor_type('torch.FloatTensor')
    _run(train_df)
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=1, start_method='fork')


num_train_steps = 8163, world_size=1
STARTING TRAINING ...

EPOCH 1
Train bi=0, F1=0.183,time=18.50462555885315
Train bi=100, F1=0.0,time=192.1209590435028
Train bi=200, F1=0.154,time=272.8310933113098
Train bi=300, F1=0.5,time=353.6001274585724
Train bi=400, F1=0.629,time=434.31355834007263
Train bi=500, F1=0.348,time=515.1855850219727
Train bi=600, F1=0.645,time=596.0435969829559
Train bi=700, F1=0.5,time=677.2618811130524
Train bi=800, F1=0.538,time=758.032787322998
Train bi=900, F1=0.455,time=838.7255313396454
Train bi=1000, F1=0.615,time=919.6912567615509
Train bi=1100, F1=0.5,time=1000.7273721694946
Train bi=1200, F1=0.606,time=1081.6975648403168
Train bi=1300, F1=0.519,time=1162.720428943634
Train bi=1400, F1=0.636,time=1243.7343821525574
Train bi=1500, F1=0.667,time=1324.761795759201
Train bi=1600, F1=0.556,time=1405.7178900241852
Train bi=1700, F1=0.516,time=1486.4517152309418
Train bi=1800, F1=0.667,time=1567.4194684028625
Train bi=1900, F1=0.6,time=1648.4677362442017
Train 

SystemExit: 0

In [14]:
val_f1s = [0] + [torch.load('val_f1_{}.pt'.format(i)) for i in range(EPOCHS)]
train_f1s = [0] + [torch.load('train_f1_{}.pt'.format(i)) for i in range(EPOCHS)]
val_losses = [0.25] + [torch.load('val_loss_{}.pt'.format(i)) for i in range(EPOCHS)]
train_losses = [0.25] + [torch.load('train_loss_{}.pt'.format(i)) for i in range(EPOCHS)]

In [15]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(1, len(val_losses)+1),
                         y=val_losses, mode="lines+markers", name="val",
                         marker=dict(color="indianred", line=dict(width=.5,
                                                                  color='rgb(0, 0, 0)'))))

fig.add_trace(go.Scatter(x=np.arange(1, len(train_losses)+1),
                         y=train_losses, mode="lines+markers", name="train",
                         marker=dict(color="darkorange", line=dict(width=.5,
                                                                   color='rgb(0, 0, 0)'))))

fig.update_layout(xaxis_title="Epochs", yaxis_title="Binary Cross Entropy",
                  title_text="Binary Cross Entropy vs. Epochs", template="plotly_white", paper_bgcolor="#f0f0f0")

fig.show()

In [16]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(1, len(val_f1s)+1),
                         y=val_f1s, mode="lines+markers", name="val",
                         marker=dict(color="indianred", line=dict(width=.5,
                                                                  color='rgb(0, 0, 0)'))))

fig.add_trace(go.Scatter(x=np.arange(1, len(train_f1s)+1),
                         y=train_f1s, mode="lines+markers", name="train",
                         marker=dict(color="darkorange", line=dict(width=.5,
                                                                   color='rgb(0, 0, 0)'))))

fig.update_layout(xaxis_title="Epochs", yaxis_title="F1 Score",
                  title_text="F1 Score vs. Epochs", template="plotly_white", paper_bgcolor="#f0f0f0")

fig.show()

In [17]:
model = CustomRoberta()
model.load_state_dict(torch.load('insincerity_model.pt'))

<All keys matched successfully>

In [18]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [19]:
device = xm.xla_device()
model = model.to(device)
model = model.eval()
def predict_insincerity(question_text, max_length):
    question_text = str(question_text)
    question_text = " ".join(question_text.split())
    inputs = tokenizer.encode_plus(
        question_text,
        None,
        add_special_tokens=True,
        max_length=max_length,
        truncation_strategy="longest_first",
        pad_to_max_length=True,
    )

    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    ids = torch.LongTensor(ids)
    mask = torch.LongTensor(mask)
    ids = torch.reshape(ids, (1,64))
    mask = torch.reshape(mask, (1,64))
    #print(ids.shape)
    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    outputs = model(
            input_ids=ids,
            attention_mask=mask,
        )
    ins = {0: 'sincere', 1: 'insincere'}
    output = ins[int(np.round(nn.Sigmoid()(outputs.detach().cpu()).item()))][0:]
    return {
        output
    }

In [20]:
predict_insincerity("Why is that stupid man the biggest dictator in the world?",64)

{'insincere'}

In [21]:
predict_insincerity("How can I train roBERTa base on TPUs?",64)

{'sincere'}

In [22]:
predict_insincerity("how many days are there in a week ?",64)

{'sincere'}

In [23]:
predict_insincerity("any women in quora had a love at first sight ?",64)

{'sincere'}